# Magic Personality Matcher

Se busca predecir la personalidad del autor de un texto, es decir, se usa texto como entrada y la salida es una de las 16 personalidades de Myers-Briggs.

Lo primero es cargar el dataset usando Pandas, este contiene solo dos columnas, personalidad y post que corresponden a la personalidad y el contenido de un post de X.

Se crea una bolsa de palabras utilizando TfidVectorizer, que se basa en la frecuencia de las palabras para determinar su importancia.

In [22]:
import joblib as jb
import numpy as np

X_train = jb.load("train_textsBoW.lzma")
y_train = jb.load("y_train.lzma")

Para la clasificación usamos el modelo Random Forest Classifier, que explorando distintos notebooks en Kaggle resultó ser el más preciso frente a otros clasificadores.

In [20]:
from sklearn.ensemble import RandomForestClassifier

svc = RandomForestClassifier(n_estimators=100, random_state=42)

svc.fit(X_train, y_train)

RandomForestClassifier(random_state=42)


Creamos un subconjunto de datos a partir de un DataFrame mas grande, en concreto, de las últimas 1000 filas del dataframe


In [ ]:
test_textsBoW = joblib.load("/content/drive/MyDrive/Colab Notebooks/preprocesado/test_textsBoW.lzma")
y_test = joblib.load("/content/drive/MyDrive/Colab Notebooks/preprocesado/y_test.lzma")

Realizamos algunas operaciones de preprocesamiento de texto en un conjunto de datos de prueba utilizando la librería NLTK

Aplicamos un modelo previamente creado para transformar datos de texto preprocesados  en un formato de Bolsa de Palabras

Utilizaremos un conjunto de datos de prueba llamado textsBowTest que contiene características como una bolsa de palabras


In [ ]:
predictions = svc.predict(test_textsBoW)

Utilizamos la funcion classification_report para generar un reporte de clasificación detallado en función de las predicciones realizadas por el modelo

In [ ]:
from sklearn.metrics import classification_report

Y_test = y_test

print(classification_report(Y_test,predictions))

              precision    recall  f1-score   support

        ENFJ       1.00      0.03      0.06       449
        ENFP       0.81      0.03      0.05      1549
        ENTJ       0.88      0.04      0.08       682
        ENTP       0.67      0.06      0.12      2563
        ESFJ       1.00      0.04      0.08        74
        ESFP       1.00      0.01      0.02       110
        ESTJ       0.89      0.39      0.54       124
        ESTP       0.97      0.40      0.56       460
        INFJ       0.41      0.50      0.45      3488
        INFP       0.39      0.29      0.33      2977
        INTJ       0.38      0.52      0.44      4863
        INTP       0.36      0.72      0.48      5438
        ISFJ       1.00      0.01      0.02       227
        ISFP       1.00      0.02      0.04       321
        ISTJ       1.00      0.01      0.02       350
        ISTP       1.00      0.02      0.04       836

    accuracy                           0.39     24511
   macro avg       0.80   

El código utiliza la librería joblib para guardar objetos de Python en archivos para su posterior carga y uso, en este caso se guardará el modelo

In [ ]:
from joblib import dump

dump(svc, "/content/drive/MyDrive/Colab Notebooks/svc.joblib")

['/content/drive/MyDrive/Colab Notebooks/svc.joblib']

In [2]:
from joblib import load

svc = load("/content/drive/MyDrive/Colab Notebooks/rfc1.joblib")
